<center><h1>Multivariate K-Nearest Neighbors</h1></center>

## 1. Recap.

we explored how to use a simple k-nearest neighbors machine learning model that used just one feature, or attribute, of the listing to predict the rent price. We first relied on the `accommodates` column, which describes the number of people a living space can comfortably accommodate. Then, we switched to the `bathrooms` column and observed an improvement in accuracy. While these were good features to become familiar with the basics of machine learning, it's clear that using just a single feature to compare listings doesn't reflect the reality of the market. An apartment that can accommodate 4 guests in a popular part of Washington D.C. will rent for much higher than one that can accommodate 4 guests in a crime ridden area.

There are 2 ways we can tweak the model to try to improve the accuracy (decrease the RMSE during validation):

- increase the number of attributes the model uses to calculate similarity when ranking the closest neighbors
- increase `k`, the number of nearby neighbors the model uses when computing the prediction

Now, we'll focus on increasing the number of attributes the model uses. When selecting more attributes to use in the model, we need to watch out for columns that don't work well with the distance equation. This includes columns containing:

- non-numerical values (e.g. city or state)
    - Euclidean distance equation expects numerical values
- missing values
    - distance equation expects a value for each observation and attribute
- non-ordinal values (e.g. latitude or longitude)
    - ranking by Euclidean distance doesn't make sense if all attributes aren't ordinal

we've read the `dc_airbnb.csv` dataset from the last lesson into pandas and brought over the data cleaning changes we made. Let's first look at the first row's values to identify any columns containing non-numerical or non-ordinal values. In the next cell, we'll drop those columns and then look for missing values in each of the remaining columns.

In [4]:
import numpy as np
import pandas as pd


In [5]:
dc_listings = pd.read_csv('dc_airbnb.csv')

In [6]:
print(np.random.default_rng(2021).permutation(dc_listings.index.size))

[ 604 2493  904 ... 1769 1333 3412]


In [7]:
dc_listings = dc_listings.loc[np.random.default_rng(2021).permutation(dc_listings.index.size)]

In [8]:
dc_listings['price'] = dc_listings['price'].str.replace('[\$,]', '', regex=True).astype('float')

### Exercise
- Use the `DataFrame.info()` method to return the number of non-null values in each column.

In [9]:
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3723 entries, 604 to 3412
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    3289 non-null   object 
 1   host_acceptance_rate  3109 non-null   object 
 2   host_listings_count   3723 non-null   int64  
 3   accommodates          3723 non-null   int64  
 4   room_type             3723 non-null   object 
 5   bedrooms              3702 non-null   float64
 6   bathrooms             3696 non-null   float64
 7   beds                  3712 non-null   float64
 8   price                 3723 non-null   float64
 9   cleaning_fee          2335 non-null   object 
 10  security_deposit      1426 non-null   object 
 11  minimum_nights        3723 non-null   int64  
 12  maximum_nights        3723 non-null   int64  
 13  number_of_reviews     3723 non-null   int64  
 14  latitude              3723 non-null   float64
 15  longitude          

## 2. Removing features.

The following columns contain non-numerical values:

- `room_type`: e.g. `Private room`
- `city`: e.g. `Washington`
- `state`: e.g. `DC`

while these columns contain numerical but non-ordinal values:

- `latitude`: e.g. 38.913458
- `longitude`: e.g. -77.031
- `zipcode`: e.g. 20009

Geographic values like these aren't ordinal, because a smaller numerical value doesn't directly correspond to a smaller value in a meaningful way. For example, the zip code 20009 isn't smaller or larger than the zip code 75023 and instead both are unique, identifier values. Latitude and longitude value pairs describe a point on a geographic coordinate system and different equations are used in those cases (e.g. haversine).

While we could convert the `host_response_rate` and `host_acceptance_rate` columns to be numerical (right now they're object data types and contain the % sign), these columns describe the host and not the living space itself. Since a host could have many living spaces and we don't have enough information to uniquely group living spaces to the hosts themselves, let's avoid using any columns that don't directly describe the living space or the listing itself:

- `host_response_rate`
- `host_acceptance_rate`
- `host_listings_count`

Let's remove these 9 columns from the Dataframe.


### Exercise

- Remove the 9 columns we discussed above from `dc_listings`:
    - 3 containing non-numerical values
    - 3 containing numerical but non-ordinal values
    - 3 describing the host instead of the living space itself

In [10]:
dc_listings.columns.values.tolist()

['host_response_rate',
 'host_acceptance_rate',
 'host_listings_count',
 'accommodates',
 'room_type',
 'bedrooms',
 'bathrooms',
 'beds',
 'price',
 'cleaning_fee',
 'security_deposit',
 'minimum_nights',
 'maximum_nights',
 'number_of_reviews',
 'latitude',
 'longitude',
 'city',
 'zipcode',
 'state']

In [11]:
columns_to_remove = [
    'host_response_rate',
    'host_acceptance_rate',
    'host_listings_count',
    'room_type',
    'latitude',
    'longitude',
    'city',
    'zipcode',
    'state',
]

dc_listings.drop(labels=columns_to_remove, axis='columns', inplace=True)

In [12]:
dc_listings.columns.values.tolist()

['accommodates',
 'bedrooms',
 'bathrooms',
 'beds',
 'price',
 'cleaning_fee',
 'security_deposit',
 'minimum_nights',
 'maximum_nights',
 'number_of_reviews']

## 3. Handling missing values.

Of the remaining columns, 3 columns have a few missing values (less than 1% of the total number of rows):

- `bedrooms`
- `bathrooms`
- `beds`

Since the number of rows containing missing values for one of these 3 columns is low, we can select and remove those rows without losing much information. There are also 2 columns that have a large number of missing values:

- `cleaning_fee` - 37.3% of the rows
- `security_deposit` - 61.7% of the rows

and we can't handle these easily. We can't just remove the rows containing missing values for these 2 columns because we'd miss out on the majority of the observations in the dataset. Instead, let's remove these 2 columns entirely from consideration.

### Exercise

- Drop the `cleaning_fee` and `security_deposit` columns from `dc_listings`.
- Then, remove all rows that contain a missing value for the `bedrooms`, `bathrooms`, or `beds` column from `dc_listings`.
    - You can accomplish this by using the [Dataframe method `dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) and setting the `axis` parameter to `0`.
    - Since only the `bedrooms`, `bathrooms`, and `beds` columns contain any missing values, rows containing missing values in these columns will be removed.
- Display the null value counts for the updated `dc_listings` Dataframe to confirm that there are no missing values left.

In [15]:
columns_missing_values = dc_listings.isnull().any()[dc_listings.isnull().any().values].index.values.tolist()

In [16]:
dc_listings[columns_missing_values].isnull().sum()

bedrooms              21
bathrooms             27
beds                  11
cleaning_fee        1388
security_deposit    2297
dtype: int64

In [17]:
print(len(dc_listings))
print(dc_listings.index.size)
print(dc_listings.shape[0])

3723
3723
3723


In [18]:
(dc_listings[columns_missing_values].isnull().sum() / len(dc_listings)).sort_values() * 100

beds                 0.295461
bedrooms             0.564061
bathrooms            0.725222
cleaning_fee        37.281762
security_deposit    61.697556
dtype: float64

In [19]:
dc_listings.drop(labels=['cleaning_fee', 'security_deposit'], axis='columns', inplace=True)

In [20]:
dc_listings.dropna(inplace=True)

In [21]:
dc_listings.index.size

3671

In [22]:
dc_listings.isnull().any()

accommodates         False
bedrooms             False
bathrooms            False
beds                 False
price                False
minimum_nights       False
maximum_nights       False
number_of_reviews    False
dtype: bool

In [23]:
dc_listings.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
604,4,1.0,1.0,1.0,110.0,2,2,1
2493,4,2.0,1.0,2.0,160.0,7,365,9
904,1,0.0,1.0,1.0,95.0,5,1125,0
1935,2,1.0,1.0,1.0,65.0,3,21,10
544,4,1.0,1.0,2.0,265.0,1,1125,15


In [24]:
dc_listings.head().describe()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
count,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.000000
mean,3.000000,1.000000,1.0,1.400000,139.000000,3.600000,527.600000,7.000000
std,1.414214,0.707107,0.0,0.547723,78.374103,2.408319,564.161147,6.363961
min,1.000000,0.000000,1.0,1.000000,65.000000,1.000000,2.000000,0.000000
25%,2.000000,1.000000,1.0,1.000000,95.000000,2.000000,21.000000,1.000000
50%,4.000000,1.000000,1.0,1.000000,110.000000,3.000000,365.000000,9.000000
75%,4.000000,1.000000,1.0,2.000000,160.000000,5.000000,1125.000000,10.000000
max,4.000000,2.000000,1.0,2.000000,265.000000,7.000000,1125.000000,15.000000


## 4. Normalize columns.

Here's how the `dc_listings`  Dataframe looks after all the changes we made:

| accommodates | bedrooms | bathrooms | beds | price | minimum_nights | maximum_nights | number_of_reviews |
|--------------|----------|-----------|------|-------|----------------|----------------|-------------------|
| 2            | 1        | 1         | 1    | 90    | 3              | 10             | 7                 |
| 9            | 3        | 2         | 3    | 225   | 3              | 1125           | 2                 |
| 3            | 1        | 1         | 2    | 95    | 2              | 700            | 14                |
| 2            | 1        | 1         | 1    | 79    | 1              | 90             | 20                |
| 2            | 1        | 1         | 1    | 45    | 2              | 1125           | 1                 |

You may have noticed that while the `accommodates`, `bedrooms`, `bathrooms`, `beds`, and `minimum_nights` columns hover between 0 and 12 (at least in the first few rows), the values in the `maximum_nights` and number_of_reviews columns span much larger ranges. For example, the maximum_nights column has values as low as 10 and as high as 1125, in the first few rows itself. If we use these 2 columns as part of a k-nearest neighbors model, these attributes could end up having an outsized effect on the distance calculations, because of the largeness of the values.

For example, 2 living spaces could be identical across every attribute but be vastly different just on the maximum_nights column. If one listing had a maximum_nights value of 1825 and the other a maximum_nights value of 4, because of the way Euclidean distance is calculated, these listings would be considered very far apart because of the outsized effect the largeness of the values had on the overall Euclidean distance. To prevent any single column from having too much of an impact on the distance, we can normalize all of the columns to have a mean of 0 and a standard deviation of 1.

Normalizing the values in each column to the [standard normal distribution](https://en.wikipedia.org/wiki/Normal_distribution#Standard_normal_distribution) (mean of 0, standard deviation of 1) preserves the distribution of the values in each column while aligning the scales. To normalize the values in a column to the standard normal distribution, you need to:

- from each value, subtract the mean of the column
- divide each value by the standard deviation of the column

Here's the mathematical formula describing the transformation that needs to be applied for all values in a column:

$$z = \frac{x- \mu}{\sigma}$$

where $x$ is a value in a specific column, $\mu$ is the mean of all the values in the column, and $\sigma$ is the standard deviation of all the values in the column. Here's what the corresponding code, using pandas, looks like:

In [25]:
# Subtract each value in the column by the mean.
first_transform = dc_listings['maximum_nights'] - dc_listings['maximum_nights'].mean()
# Divide each value in the column by the standard deviation.
normalized_col = first_transform / first_transform.std()

It should be noted that you can also do the following:

In [26]:
normalized_col = first_transform / dc_listings['maximum_nights'].std()

and get the same answer as above.

This is because `first_transform` is merely shifting the mean of the distribution and has no effect on the shape or scaling of the distribution. In other words, the variance of `dc_listings` is the same as the variance of first_transform.

To apply this transformation across all of the columns in a Dataframe, you can use the corresponding Dataframe methods `mean()` and `std()`:

```python
normalized_listings = (dc_listings - dc_listings.mean()) / (dc_listings.std())
```


Copy
These methods were written with mass column transformation in mind and when you call mean() or std(), the appropriate column means and column standard deviations are used for each value in the Dataframe. Let's now normalize all of the feature columns in dc_listings.

### Exercise

- Normalize all of the feature columns in `dc_listings` and assign the new Dataframe containing just the normalized feature columns to `normalized_listings`.

- Add the `price` column from `dc_listings` to `normalized_listings`.
- Display the first 3 rows in `normalized_listings`.

In [27]:
normalized_listings = (dc_listings - dc_listings.mean()) / dc_listings.std()
normalized_listings.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
604,0.401366,-0.249467,-0.439151,-0.546858,-0.282396,-0.065038,-0.016604,-0.482505
2493,0.401366,0.939875,-0.439151,0.297345,0.081108,1.316644,-0.016594,-0.208875
904,-1.095499,-1.438810,-0.439151,-0.546858,-0.391448,0.763971,-0.016573,-0.516709
1935,-0.596544,-0.249467,-0.439151,-0.546858,-0.609550,0.211298,-0.016604,-0.174671
544,0.401366,-0.249467,-0.439151,0.297345,0.844466,-0.341375,-0.016573,-0.003652


In [28]:
normalized_listings['price'] = dc_listings['price']
normalized_listings.head(3)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
604,0.401366,-0.249467,-0.439151,-0.546858,110.0,-0.065038,-0.016604,-0.482505
2493,0.401366,0.939875,-0.439151,0.297345,160.0,1.316644,-0.016594,-0.208875
904,-1.095499,-1.438810,-0.439151,-0.546858,95.0,0.763971,-0.016573,-0.516709


## 5. Euclidean distance for multivariate case.

In the last lesson, we trained 2 univariate k-nearest neighbors models. The first one used the accommodates attribute while the second one used the bathrooms attribute. Let's now train a model that uses both attributes when determining how similar 2 living spaces are. Let's refer to the Euclidean distance equation again to see what the distance calculation using 2 attributes would look like:

$$$$

Since we're using 2 attributes, the distance calculation would look like:

$$$$

To find the distance between 2 living spaces, we need to calculate the squared difference between both accommodates values, the squared difference between both bathrooms values, add them together, and then take the square root of the resulting sum. Here's what the Euclidean distance between the first 2 rows in normalized_listings looks like:

<img src="figs/distance_two_features.png" width="600" heigth="400"/>

So far, we've been calculating Euclidean distance ourselves by writing the logic for the equation ourselves. We can instead use the [distance.euclidean() function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html) from `scipy.spatial`, which takes in 2 vectors as the parameters and calculates the Euclidean distance between them. The `euclidean()` function expects:

- both of the vectors to be represented using a **list-like** object (Python list, NumPy array, or pandas Series)
- both of the vectors must be 1-dimensional and have the same number of elements

Here's a simple example:

In [29]:
from scipy.spatial import distance

first_listing = [-0.596544, -0.439151]
second_listing = [-0.596544, 0.412923]
dist = distance.euclidean(first_listing, second_listing)

print(dist)

0.852074


Let's use the `euclidean()` function to calculate the Euclidean distance between 2 rows in our dataset to practice.

### Exercise

- Calculate the Euclidean distance using only the `accommodates` and `bathrooms` features between the first row and fifth row in `normalized_listings` using the `distance.euclidean()` function.

- Assign the distance value to `first_fifth_distance` and display using the `print` function.

In [30]:
normalized_listings[['accommodates', 'bathrooms']].head()

,accommodates,bathrooms
604,0.401366,-0.439151
2493,0.401366,-0.439151
904,-1.095499,-0.439151
1935,-0.596544,-0.439151
544,0.401366,-0.439151


In [31]:
first_fifth_distance = distance.euclidean(normalized_listings.iloc[0][['accommodates', 'bathrooms']], normalized_listings.iloc[4][['accommodates', 'bathrooms']])
print(first_fifth_distance)

0.0


## 6. Introduction to scikit-learn.

So far, we've been writing functions from scratch to train the k-nearest neighbor models. While this is helpful deliberate practice to understand how the mechanics work, you can be more productive and iterate quicker by using a library that handles most of the implementation. We'll learn about the [scikit-learn library](https://scikit-learn.org/stable/), which is the most popular machine learning library in Python. Scikit-learn contains functions for all of the major machine learning algorithms and a simple, unified workflow. Both of these properties allow data scientists to be incredibly productive when training and testing different models on a new dataset.

The scikit-learn workflow consists of 4 main steps:

- instantiate the specific machine learning model you want to use
- fit the model to the training data
- use the model to make predictions
- evaluate the accuracy of the predictions


We'll focus on the first 3 steps in this cell and the next cell. Each model in scikit-learn is implemented as a [separate class](https://scikit-learn.org/stable/modules/classes.html) and the first step is to identify the class we want to create an instance of. In our case, we want to use the [KNeighborsRegressor class](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor).

Any model that helps us predict numerical values, like listing price in our case, is known as a `regression` model. The other main class of machine learning models is called `classification`, where we're trying to predict a label from a fixed set of labels (e.g. blood type or gender). The word **regressor** from the class name `KNeighborsRegressor` refers to the regression model class that we just discussed.

Scikit-learn uses a similar object-oriented style to Matplotlib and you need to instantiate an empty model first by calling the constructor:

```python
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
```

If you refer to the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor), you'll notice that by default:

- `n_neighbors`: the number of neighbors, is set to `5`
- `algorithm`: for computing nearest neighbors, is set to `auto`
- `p`: set to 2, corresponding to Euclidean distance

Let's set the `algorithm` parameter to `brute` and leave the `n_neighbors` value as `5`, which matches the implementation we wrote in the last lesson. If we leave the `algorithm` parameter set to the default value of `auto`, scikit-learn will try to use tree-based optimizations to improve performance (which are outside of the scope of this lesson):

```python
knn = KNeighborsRegressor(algorithm='brute')
```


## 7. Fitting a model and making predictions.

Now, we can fit the model to the data using the fit method. For all models, the fit method takes in 2 required parameters:

matrix-like object, containing the feature columns we want to use from the training set.
list-like object, containing correct target values.
Matrix-like object means that the method is flexible in the input and either a Dataframe or a NumPy 2D array of values is accepted. This means you can select the columns you want to use from the Dataframe and use that as the first parameter to the fit method.

If you recall from earlier in the lesson, all of the following are acceptable list-like objects:

- NumPy array
- Python list
- pandas Series object (e.g. when selecting a column)

You can select the target column from the Dataframe and use that as the second parameter to the fit method:


```python
# Split full dataset into train and test sets.
train_df = normalized_listings.iloc[0:2792]
test_df = normalized_listings.iloc[2792:]

# Matrix-like object, containing just the 2 columns of interest from training set.
train_features = train_df[['accommodates', 'bathrooms']]

# List-like object, containing just the target column, `price`.
train_target = train_df['price']

# Pass everything into the fit method.
knn.fit(train_features, train_target)
```


When the `fit()` method is called, scikit-learn stores the training data we specified within the KNearestNeighbors instance `(knn)`. If you try passing in data containing missing values or non-numerical values into the `fit` method, scikit-learn will return an error. Scikit-learn contains many such features that help prevent us from making common mistakes.

Now that we specified the training data we want used to make predictions, we can use the [predict method](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor.predict) to make predictions on the test set. The predict method has only one required parameter:

- matrix-like object, containing the feature columns from the dataset we want to make predictions on

The number of feature columns you use during both training and testing need to match or scikit-learn will return an error:

```python
predictions = knn.predict(test_df[['accommodates', 'bathrooms']])
```

The `predict()` method returns a NumPy array containing the predicted price values for the test set. You now have everything you need to practice the entire scikit-learn workflow.

### Exercise

- Create an instance of the [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor) class with the following parameters:
    - `n_neighbors`: `5`
    - `algorithm`: brute
- Use the `fit` method to specify the data we want the k-nearest neighbor model to use. Use the following parameters:
    - training data, feature columns: just the `accommodates` and `bathrooms` columns, in that order, from train_df.
    - training data, target column: the `price` column from `train_df`.
- Call the `predict` method to make predictions on:
    - the `accommodates` and `bathrooms` columns from `test_df`
    - assign the resulting NumPy array of predicted price values to `predictions`.

In [102]:
from sklearn.neighbors import KNeighborsRegressor

In [101]:
train_df = normalized_listings.iloc[0:2792]
test_df = normalized_listings.iloc[2792:]

In [103]:
kneighreg = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

In [104]:
kneighreg.fit(train_df[['accommodates', 'bathrooms']], train_df['price'])

KNeighborsRegressor(algorithm='brute')

In [105]:
predictions = kneighreg.predict(test_df[['accommodates', 'bathrooms']])
print(predictions)

[446.4 133.6 133.6 130.  299.   99.4  99.4  99.4 133.6  99.4 299.   99.4
  78.8 130.  266.6 266.6  99.4 200.2 130.   99.4  64.4  99.4  99.4 130.
  99.4 372.4  99.4  99.4 133.6  99.4  99.4 315.6  99.4 130.   99.4  99.4
 130.  145.8 130.   99.4  99.4 299.  609.8  78.8 299.  130.  266.6 299.
 270.4  99.4  99.4 299.  130.   99.4 152.4  99.4  99.4 138.6 102.6 130.
  78.8 299.  128.6  99.4  99.4 299.  266.6 261.2  99.4  99.4 266.6 226.4
 138.6 130.  130.   99.4 133.6 130.  130.   99.4 164.8  99.4 299.  260.2
 266.6 299.   99.4 133.6  99.4 130.   99.4  99.4  99.4 200.2 138.6 320.
 130.   99.4 372.4 200.2 130.  130.  130.  102.6 102.6  99.4 130.  299.
 145.8 179.  133.6 270.4 200.2  77.   77.  130.   99.4 270.4  99.4 938.8
 200.2 133.6 130.  299.  130.   99.4 299.  130.  265.8 315.6 133.6  99.4
  99.4  99.4  99.4  78.8 133.6 130.  102.6 372.4 138.6 133.6  99.4 133.6
  99.4 114.   99.4 130.   99.4 200.2  99.4 270.4  99.4 145.8 226.4 102.6
 270.4  99.4 145.8 133.6 133.6 130.   67.8 133.6  99.4 1

## 8. Calculating MSE using Scikit-Learn.

Earlier in this lesson, we calculated the MSE and RMSE values using the pandas arithmetic operators to compare each predicted value with the actual value from the `price` column of our test set. Alternatively, we can instead use the [sklearn.metrics.mean_squared_error function()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html). Once you become familiar with the different machine learning concepts, unifying your workflow using scikit-learn helps save you a lot of time and avoid mistakes.

The `mean_squared_error()` function takes in 2 inputs:

- list-like object, representing the true values
- list-like object, representing the predicted values using the model

For this function, we won't show any sample code and will leave it to you to understand the function [from the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) itself to calculate the MSE and RMSE values for the predictions we just made.

### Exercise

- Use the `mean_squared_error` function to calculate the MSE value for the predictions we made in the previous cell.
- Assign the MSE value to `two_features_mse`.
- Calculate the RMSE value by taking the square root of the MSE value and assign to `two_features_rmse`.
- Display both of these error scores using the `print` function.

In [107]:
train_columns = ['accommodates', 'bathrooms']
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute', metric='euclidean')
knn.fit(train_df[train_columns], train_df['price'])
predictions = knn.predict(test_df[train_columns])

In [108]:
from sklearn.metrics import mean_squared_error

In [109]:
two_features_mse = mean_squared_error(test_df['price'], predictions)
two_features_rmse = np.sqrt(two_features_mse)

In [110]:
print(two_features_mse)
print(two_features_rmse)

10880.390125142207
104.30910854351218


## 9. Using more features.

Here's a table comparing the MSE and RMSE values for the 2 univariate models from the last lesson and the multivariate model we just trained:

| feature(s)              | MSE     | RMSE  |
|-------------------------|---------|-------|
| accommodates            | 18646.5 | 136.6 |
| bathrooms               | 17333.4 | 131.7 |
| accommodates, bathrooms | 10880.4 | 104.1 |


As you can tell, the model we trained using both features ended up performing better (lower error score) than either of the univariate models from the last lesson. Let's now train a model using the following 4 features:

- accommodates
- bedrooms
- bathrooms
- number_of_reviews

Scikit-learn makes it incredibly easy to swap the columns used during training and testing. We're going to leave this for you as a challenge to train and test a k-nearest neighbors model using these columns instead. Use the code you wrote in the last cell as a guide.

### Exercise

- Create a new instance of the KNeighborsRegressor class with the following parameters:
    - `n_neighbors`: `5`
    - `algorithm`: `brute`

- Fit a model that uses the following columns from our training set (`train_df`):
    - accommodates
    - bedrooms
    - bathrooms
    - number_of_reviews

- Use the model to make predictions on the test set (`test_df`) using the same columns. Assign the NumPy array of predictions to `four_predictions`.
- Use the `mean_squared_error()` function to calculate the MSE value for these predictions by comparing `four_predictions` with the `price` column from `test_df`. Assign the computed MSE value to `four_mse`.
- Calculate the RMSE value and assign to `four_rmse`.
- Display `four_mse` and `four_rmse` using the `print` function.

In [ ]:
features = ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews']
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

In [112]:
knn.fit(train_df[features], train_df['price'])

KNeighborsRegressor(algorithm='brute')

In [113]:
four_predictions = knn.predict(test_df[features])
four_mse = mean_squared_error(test_df['price'], four_predictions)
four_rmse = np.sqrt(four_mse)


In [114]:
print(four_mse)
print(four_rmse)

8789.460978384526
93.75212519396308


## 10. Using all features.
### Exercise

- Use all of the columns, except for the `price` column, to train a k-nearest neighbors model using the same parameters for the `KNeighborsRegressor` class as the ones from the last few cells.
- Use the model to make predictions on the test set and assign the resulting NumPy array of predictions to `all_features_predictions`.
- Calculate the MSE and RMSE values and assign to all_features_mse and all_features_rmse accordingly.
- Use the print function to display both error scores.

In [121]:
test_df.columns

Index(['accommodates', 'bedrooms', 'bathrooms', 'beds', 'price',
       'minimum_nights', 'maximum_nights', 'number_of_reviews'],
      dtype='object')

In [ ]:
all_features = [
    'accommodates',
     'bedrooms',
     'bathrooms',
     'beds',
#      'price',
     'minimum_nights',
     'maximum_nights',
     'number_of_reviews',
]

In [116]:
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')
knn.fit(train_df[all_features], train_df['price'])

KNeighborsRegressor(algorithm='brute')

In [118]:
all_features_predicitions = knn.predict(test_df[all_features])

In [119]:
all_features_mse = mean_squared_error(test_df['price'], all_features_predicitions)
all_features_rmse = np.sqrt(all_features_mse)

In [120]:
print(all_features_mse)
print(all_features_rmse)

7377.0400910125145
85.88969723437448
